# Driver Drowsiness Detection System

Studies indicate that fatigue-related crashes account for about 20% of road accidents and even more on roads with no driving hour regulations. Driver detection systems, particularly those focusing on drowsiness detection, aim to address that alarming rate by monitoring drivers for signs of drowsiness and issuing timely alerts to prevent potential crashes.

For our final project, we chose to develop a DDS that will utilize two two convolutional neural networks (CNN). One will be trained on the UTA Dataset (comprised of drowsy/neutral/alert driver images), and the other on Behavioral Signs Dataset (containing images of closed/open eyes, and yawning/not yawning drivers); both model's predictions will be layer combined to output the final result. The combined model will then provide warnings and alerts based on the detected level of fatigue in real time. 

### Requirements 
- TensorFlow: Developed by the Google Brain team for machine learning and artificial intelligence, Tensorflow has a allows for training and inference of deep neural networks.

- Keras: Provides a Python interface for artificial neural networks (inbuilt python library).

- Numpy: Used for scientific computing in Python. Provides support for arrays, matrices, and various mathematical functions to operate on them. 

- OpenCV: Machine learning and compiter vision library; contains >2500 algorhitms optimized for various CV tasks; allows for webcam access.

- Scikit-learn: Data mining, data analysis. In this project, used for splitting datasets. 

- Pandas: Data manipulation and analysis library. Used to create dataframes associating frames with their labels.

- Kagglehub: For downloading Kaggle datasets 

- Visualkeras: For network visualization


In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import cv2
import os
import kagglehub
import visualkeras


In [54]:
# PRESETS 
# Current working directory 
working_dir = os.getcwd()
# UTA DD Dataset directory 
uta_dir = working_dir + '/frames'
# Behavioral Signs Dataset directory
behavior_dir = working_dir + '/behavioral_signs_frames'
# OpenCV cascade for face detection
face_cascade_path = working_dir + '/haarcascade_frontalface_default.xml'
# Image size for VGG16
IMG_SIZE = 224 
# Initializing face cascade (will be used to detect faces in the images)
face_cascade = cv2.CascadeClassifier(face_cascade_path)
# Initializing the VGG16 model (will be used for feature extraction, not the final prediction)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

## Frame - Class (Label) Association
### Appending Labels for the UTA Dataset 
Frames captured are associated with **0 = not drowsy or neutral, 1 = drowsy classes**, based on the 'vid' label within the parsed filename. They're then processed (faces of drivers are extracted and saved as pixel values) and added to a pandas dataframe for training, validating, and testing. 

In [126]:
def parse_filename(filename):
    parts = filename.split('_')
    for i, part in enumerate(parts):
        if part.lower() == 'vid':
            label = int(parts[i + 1])
            if label == 0 or label == 5:
                return 0
            elif label == 10: 
                return 1
            else:
                return None
    return None

In [127]:
# Function to process the images, detect faces, crop, and use VGG16 for feature extraction.
# Returns the extracted features and labels. 
def extract_features_uta(image_dir, face_cascade, vgg16_model):
    data = []
    labels = []  
    # Iterating through the image directories
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith('.jpg'):  
                img_path = os.path.join(root, file)
                
                img = cv2.imread(img_path)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                # Detecting faces
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                
                # Processing each detected face
                for (x, y, w, h) in faces:
                    face = img[y:y+h, x:x+w]  # Cropping the face
                    # Resizing the face to the input size expected by VGG16
                    resized_face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
                    label = parse_filename(file)  
                    data.append([resized_face, label])
    return pd.DataFrame(data, columns=['features', 'label'])
    
df_drowsiness = extract_features_uta(uta_dir, face_cascade, vgg16_model)

# Sanity check
df_drowsiness.sample(5)

,features,label
239,"[[[117, 136, 151], [117, 136, 151], [117, 136,...",0
2,"[[[30, 37, 46], [31, 38, 46], [31, 37, 43], [3...",0
820,"[[[53, 46, 43], [54, 46, 45], [54, 46, 46], [5...",0
29,"[[[82, 108, 122], [83, 109, 123], [83, 108, 12...",1
53,"[[[110, 124, 136], [118, 132, 144], [118, 132,...",1


### Appending Labels for the Behavioral Signs Dataset
To adjust the complexity of our final (combined) model, we will train a second CNN that utilizes the Behavioral Signs Dataset. The behavioral_signs_frames folder contains labeled images of open and closed eyes, as well as yawning and non-yawning drivers. This added data will significantly enhance the model's ability to generalize, addressing the overfitting issue encountered when using only the UTA dataset frames in our initial model (will be used as features in the final model)

In [165]:
label_map = {'closed': 0, 'open': 1, 'no_yawn': 2, 'yawn': 3}

# Function to preprocess images and detect faces from the behavioral signs dataset. Returns a DataFrame with the processed data.
def extract_features_behavior(image_dir, face_cascade_path, label_map):
    data = []
    # No need to extract faces in the case of 'open' and 'closed' labels
    for label in os.listdir(image_dir):
        # Constructing the path to the label folder 
        label_folder_path = os.path.join(image_dir, label)
        if label in ['open', 'closed']:  
            if os.path.isdir(label_folder_path):
                for image_name in os.listdir(label_folder_path):
                    img_array = cv2.imread(os.path.join(label_folder_path, image_name), cv2.IMREAD_COLOR)
                    array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    data.append([array, label_map[label]])
        else: # if label is 'yawn' or 'no_yawn'
            if os.path.isdir(label_folder_path):
                for image_name in os.listdir(label_folder_path):
                    image_path = os.path.join(label_folder_path, image_name)

                    # Load the image, skip if invalid
                    img = cv2.imread(image_path)
                    if img is None:
                        continue 

                    # Convert to grayscale (required for face detection)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

                    # If a face is detected, crop and resize the face
                    for (x, y, w, h) in faces:
                        face = img[y:y+h, x:x+w]  # Cropping the face
                        resized_face = cv2.resize(face, (IMG_SIZE, IMG_SIZE)) 
                        behavior_label = label_map[label]
                        data.append([resized_face, behavior_label])
    return pd.DataFrame(data, columns=['features', 'behavior_label'])

# Create the DataFrame with face detection and preprocessing
df_behavior = extract_features_behavior(behavior_dir, face_cascade_path, label_map) # Has shape (224, 224, 3)

# Sanity check 
df_behavior.sample(5)

,features,behavior_label
409,"[[[26, 32, 55], [26, 32, 55], [27, 33, 56], [2...",0
1168,"[[[57, 63, 52], [57, 63, 52], [57, 63, 52], [5...",3
1320,"[[[157, 151, 138], [158, 152, 139], [159, 156,...",1
820,"[[[76, 91, 94], [40, 53, 57], [71, 84, 87], [1...",2
1681,"[[[5, 13, 26], [7, 15, 28], [10, 18, 32], [17,...",1


## Data Preparation and Augmentation 
### UTA Model
The UTA RealLife Drowsiness Dataset is split into training, validation, and testing sets, and later converted to tensorflow datasets. The frames have been previously rescaled and augmented. 

In [128]:
# Splittig based on the label. 30% test, 25% validation, 45% training
train_val_df, test_df = train_test_split(df_drowsiness, test_size=0.3, stratify=df_drowsiness['label'], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['label'], random_state=42)

# Converting DataFrame columns to TensorFlow datasets
def df_to_dataset(df, batch_size=32):
  features = np.stack(df['features'].values)
  labels = df['label'].values
  dataset = tf.data.Dataset.from_tensor_slices((features, labels))
  dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

# Creating TensorFlow datasets
train_dataset = df_to_dataset(train_df)
val_dataset = df_to_dataset(val_df)
test_dataset = df_to_dataset(test_df)

### Behavioral Model
The behavioral model is split into datasets as the UTA model, with the features converted into numPy arrays to match the expected network input. 

In [167]:
# Splitting the data into features and labels
X = np.stack(df_behavior['features'].values)  # Converting features to a numpy array. Will have shape (tot_imgs, 224, 224, 3)
y = np.stack(df_behavior['behavior_label'].values) # Converting labels to a numpy array. Will have shape (tot_imgs,)

# Splitting into train, validation, and test sets.
X_train_beh, X_test_beh, y_train_beh, y_test_beh = train_test_split(X, y, random_state=42, test_size=0.3)
X_val_beh, X_test_beh, y_val_beh, y_test_beh = train_test_split(X_test_beh, y_test_beh, test_size=0.5, random_state=42)

# Data augmentation for improved generalization
train_behavioral = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
validate_behavioral = ImageDataGenerator(rescale=1/255)
test_behavioral = ImageDataGenerator(rescale=1/255)

# Setting the batch size for improved generalization
train_behavioral = train_behavioral.flow(np.array(X_train_beh), y_train_beh, shuffle=False)
validate_behavioral = validate_behavioral.flow(np.array(X_val_beh), y_val_beh, shuffle=False)
test_behavioral = test_behavioral.flow(np.array(X_test_beh), y_test_beh, shuffle=False, batch_size=32)

## Model Definition, Compilation, and Training
### UTA Model D, C & T 
The model architecture is defined using a pre-trained (on ImageNet) VGG16 base model. The top layers are excluded and the input shape is specified to match the dimensions of our input data. Custom layers are then added for the 3-class classification. To prevent the weights of the pre-trained VGG16 base model from being updated during training, we freeze all the layers of the base model, after which the model is compiled, and trained using the training and validation datasets. 

In [129]:
# Loading the VGG16 model (pretrained on ImageNet)
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Adding the fully connected layers for classification
model = Sequential([
    vgg16_base,
    Flatten(), 
    Dense(256, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Regularization
    Dense(1, activation='sigmoid') # Output layer (binary classification)
])

# Freezing  the base model layers to keep the pretrained weights
for layer in vgg16_base.layers:
    layer.trainable = False

# Compiling the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Training
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    batch_size=32,
)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 163s 12s/step - accuracy: 0.8225 - loss: 3.7657 - val_accuracy: 0.9329 - val_loss: 4.5201
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 148s 11s/step - accuracy: 0.9866 - loss: 0.5575 - val_accuracy: 0.9799 - val_loss: 1.8185
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 141s 10s/step - accuracy: 0.9971 - loss: 0.0595 - val_accuracy: 0.9664 - val_loss: 1.7039
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 142s 10s/step - accuracy: 0.9995 - loss: 0.0196 - val_accuracy: 0.9799 - val_loss: 1.3128
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 138s 10s/step - accuracy: 1.0000 - loss: 6.8437e-06 - val_accuracy: 0.9866 - val_loss: 1.3736
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 148s 11s/step - accuracy: 0.9992 - loss: 0.0678 - val_accuracy: 0.9866 - val_loss: 1.3257
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 145s 11s/step - accuracy: 1.0000 - loss: 1.1004e-21 - val_accuracy: 0.9799 - val_loss: 1.3399
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 147s 11s/step - accuracy: 1.0000 - loss: 2.5186e-18 - val_

In [130]:
model.save_weights('drowsiness_weights.weights.h5')

### Behavioral Model D, C & T
The model is designed to classify images into yawning, not yawning, open eyes, and closed eyes categories. The model uses **convolutional layers** which help to learn spatial features in the images (higher num of filters in initial layers captures low-level edges and textures, while the decreasing spatial dimension deeper in the network captures more abstract features). 512 and 128 have been found to work well for image classification - VGG16 has similar configuration. **MaxPooling** reduces the spatial dimensions of the images (-> model focuses on the most important features), **dropout** helps prevent overfitting, and the final **softmax** layer outputs the probabilities for each of the four classes. Sparse categorical crossentropy automatically handles integers ranging from [0,3] to convert them to a one-hot encoded format. 
The class with the highest probability is the predicted label.

In [ ]:
model_beh = Sequential([
    Conv2D(256, (3, 3), activation='relu', input_shape=X_train_beh.shape[1:]),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

model_beh.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Since s_c_c automatically one-hot encodes the 4 labels
    metrics=['accuracy']
)

history_beh = model_beh.fit(
    train_behavioral,
    validation_data=validate_behavioral,
    epochs=20,
    shuffle=True,
)

/Users/zdzilowska/miniconda3/envs/dis/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/zdzilowska/miniconda3/envs/dis/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 278s 6s/step - accuracy: 0.3925 - loss: 1.4297 - val_accuracy: 0.7370 - val_loss: 0.6911
Epoch 2/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 298s 7s/step - accuracy: 0.5972 - loss: 0.9039 - val_accuracy: 0.7509 - val_loss: 0.5815
Epoch 3/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 314s 7s/step - accuracy: 0.6492 - loss: 0.8235 - val_accuracy: 0.8097 - val_loss: 0.4506
Epoch 4/20
37/43 ━━━━━━━━━━━━━━━━━━━━ 39s 7s/step - accuracy: 0.6727 - loss: 0.7496

In [ ]:
model_beh.save_weights('behavioral_weights.weights.h5')

## Network Architecture 
### UTA Model Architecture

In [ ]:
from keras.utils.vis_utils import plot_model
import visualkeras

plot_model(model, to_file='model_drowsiness.png', show_shapes=True, show_layer_names=True)
visualkeras.layered_view(model, legend=True, draw_volume=False).show()

### Behavioral Model Architecture

In [ ]:
plot_model(model_beh, to_file='model_behavioral.png', show_shapes=True, show_layer_names=True)
visualkeras.layered_view(model_beh, legend=True, draw_volume=False).show()

## Performance evaluation
### UTA Model Performance
With the accuracy surprisingly high, we've checked the appropriateness of labels (correct - the dataset is balanced; same amount of drowsy, and not drowsy images), and for potential data leakage issues (none - we've verified the integrity of train-test splits, all of the dataframe's feature column cells are unique).

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=tf.data.experimental.cardinality(test_dataset).numpy())
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# Accuracy and loss plots
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

# Highest testing accuracy and its index
max_val_acc = max(val_accuracy)
max_val_acc_index = val_accuracy.index(max_val_acc)
min_val_acc = min(val_accuracy)
max_accuracy = max(accuracy)
min_loss =  min(loss)
min_val_loss = min(val_loss)
plt.plot(max_val_acc_index, max_val_acc, marker='o', color='lightpink')

# Training and testing accuracy over epochs
plt.plot(epochs, accuracy, "g", label="Training Accuracy")
plt.plot(epochs, val_accuracy, "r", label="Testing Accuracy")
plt.legend()
plt.title("Training and Testing Accuracy of the UTA Model\nHighest Testing Accuracy: {:.2f}%".format(max_val_acc*100))
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

# Plotting loss over epochs
plt.plot(epochs, loss, "g", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Testing loss")
plt.legend()
plt.title("Training and Testing Loss of the UTA Model")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

8/8 ━━━━━━━━━━━━━━━━━━━━ 70s 9s/step - accuracy: 1.0000 - loss: 6.5497e-13
Test loss: 4.011881701414949e-13
Test accuracy: 1.0


### Behavioral CNN Performance

In [ ]:
accuracy = history_beh.history['accuracy']
val_accuracy = history_beh.history['val_accuracy']
loss = history_beh.history['loss']
val_loss = history_beh.history['val_loss']
epochs = range(len(accuracy))

max_val_acc = max(val_accuracy)
max_val_acc_index = val_accuracy.index(max_val_acc)
min_val_acc = min(val_accuracy)
max_accuracy = max(accuracy)
min_loss =  min(loss)
min_val_loss = min(val_loss)
plt.plot(max_val_acc_index, max_val_acc, marker='o', color='lightpurple')

plt.plot(epochs, accuracy, "g", label="Training Accuracy")
plt.plot(epochs, val_accuracy, "r", label="Testing Accuracy")
plt.legend()
plt.title("Training and Testing Accuracy of the Behavioral Model\nHighest Testing Accuracy: {:.2f}%".format(max_val_acc*100))
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

plt.plot(epochs, loss, "g", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Testing loss")
plt.legend()
plt.title("Training and Testing Loss of the Behavioral Model")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

### Real-Time Driver Drowsiness Detection 
Using the models defined above (which was ), we'll now implement a real-time DDS based on a webcam. The code captures live video frames using OpenCV, processes them to match the input requirements of the trained model, and performs predictions to determine drowsy vs alert state. The model's predictions are overlaid onto the video feed in real time, displaying the driver's status and confidence scores. 

*The system is interactive, allowing the user to view the annotated video feed and terminate the program by pressing the 'q' key.*

In [ ]:
# Rebuilding both models to omit steps above, with the weights already generated
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  
model_drowsiness = Model(inputs=base_model.input, outputs=predictions)

model_behavior = Sequential([
  Conv2D(128, (3, 3), activation='relu', input_shape=(X_train_beh.shape[1:])),  
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Flatten(),
  Dropout(0.5),
  Dense(64, activation='relu'),
  Dense(4, activation='softmax')  
])
model_behavior.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

model_drowsiness_path = "drowsiness_weights.weights.h5"
model_drowsiness.load_weights(model_drowsiness_path)

model_behavior_path = "behavioral_weights.weights.h5"
model_behavior.load_weights(model_behavior_path)

In [ ]:
# Configuration for alert system
consecutive_drowsy_frames = 30  # Num of consecutive "drowsy" frames to trigger alert
drowsy_counter = 0

# Defining the preprocessing function to match the model's input requirements. Returns a single adjusted frame. 
def preprocess_frame(frame, face_cascade=face_cascade):
  # Read in the image
  img = cv2.imread(frame, cv2.IMREAD_COLOR)
  # Normalize 
  img = img / 255
  # Resize to match the size used during training
  resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
  # Reshape the image to match the input shape 
  return resized.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# Starting the camera; 0 - default camera
cap = cv2.VideoCapture(0) 

print("Press 'q' to quit the webcam stream.")
while cap.isOpened():
  ret, frame = cap.read()  # Reading frame-by-frame
  if not ret:
    print("Failed to capture frame. Exiting...")
    break

  processed_frame = preprocess_frame(frame)

  # Making a prediction
  prediction_drowsy = model_drowsiness.predict(processed_frame)
  drowsy_label = np.argmax(prediction_drowsy) # Index of the max value in the array of probabilities - max probability class
  prediction_beh = model_behavior.predict(processed_frame)
  beh_label = np.argmax(prediction_beh)

  # Combine predictions. Final status will be drowsy if 30 cons frames are drowsy (/closed/yawn)
  if drowsy_label == 0 or (beh_label in [1, 3]):  # drowsy or closed/yawn
    drowsy_counter += 1
  else: 
    drowsy_counter = 0

  print(drowsy_counter)

  # Checking if alert conditions are met
  if drowsy_counter >= consecutive_drowsy_frames:
    status = "Drowsiness Detected"
    color = (0, 0, 255)  
  else :
    status = "No Drowsiness Detected"
    color = (0, 255, 0)

  # Annotating the frame
  cv2.putText(frame, f"Status: {status}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
  cv2.putText(frame, f"Confidence: {drowsy_prob:.2f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
  # Displaying the frame with annotations
  cv2.imshow("Drowsy Driver Detection", frame)

  # Breaking the loop once 'q' is pressed
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()
print(f"Total frames processed: {frame_count}")


Press 'q' to quit the webcam stream.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step
23
1/1 ━━

### Confusion Matrix ###
Finish later

In [ ]:
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Alert", "Drowsy"])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()